In [ ]:
import re
import statistics
from collections import defaultdict

In [ ]:
import logging
import pymystem3
from pydantic import BaseModel

mystem = pymystem3.Mystem(entire_input=False, disambiguation=True)

In [ ]:
import nltk
import json
import numpy as np
import pandas as pd
from joblib import load
from nltk.tokenize import sent_tokenize
from sklearn import linear_model
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Analyzer:

    COLUMNS_NEEDED_NATIVE = [
        "words",
        "unique_words",
        "sentences",
        "characters",
        "mean_len_word",
        "mean_len_sentence",
        "formula_flesh_oborneva",
        "formula_flesh_kinc_oborneva",
        "formula_pushkin",
        "formula_pushkin_100",
        "level_comment",
        "structure_complex",
        "lexical_complex",
        "narrativity",
        "description",
        "tt_ratio",
        "lex_density",
        "lexical_complex_rki",
        "detcorpus_5000",
        "rare_words",
        "frequency_bag",
    ]

    GRAM_FEATURES = [
        "A",
        "ADV",
        "ADVPRO",
        "ANUM",
        "APRO",
        "COM",
        "CONJ",
        "INTJ",
        "NUM",
        "PART",
        "PR",
        "S",
        "SPRO",
        "V",
        "непрош",
        "прош",
        "им",
        "пр",
        "род",
        "твор",
        "деепр",
        "изъяв",
        "инф",
        "пов",
        "прич",
        "кр",
        "полн",
        "притяж",
        "1-л",
        "сред",
        "несов",
        "сов",
        "действ",
        "страд",
        "неод",
        "од",
    ]

    # шкала сложности, высчитывается от 0 до 10, примерно соответствует классу
    INTERPRETER = [
        ("Очень простой текст, подойдет для возраста 7-8 лет (1-2 класс).", 0, 20),
        ("Простой текст, подойдет для возраста 9-10 лет (3-4 класс).", 20, 40),
        ("Достаточно простой текст, подойдет для возраста 11-12 лет (5-6 класс).", 40, 60,),
        ("Текст подойдет для возраста 13-15 лет (7-9 класс).", 60, 75),
        ("Текст подойдет для возраста 16-17 лет (10-11 класс).", 75, 85),
        ("Сложный текст, подойдет для студента ВУЗа и старше", 85, 91),
        ("Очень сложный текст, подойдет для выпускника ВУЗа и старше", 91, 100),
    ]

    def __init__(self, mystem):
      
        self.mystem = mystem

        self.ridge = linear_model.Ridge(alpha=0.1)

        self.features = pd.read_csv("/content/drive/MyDrive/data/list_of_features_1207.csv")

        # списки частотных слов
        self.fr_100_list = self.__load("/content/drive/MyDrive/data/fr_100.txt")
        self.fr_300_list = self.__load("/content/drive/MyDrive/data/fr_300.txt")
        self.fr_500_list = self.__load("/content/drive/MyDrive/data/fr_500.txt")
        self.fr_1000_list = self.__load("/content/drive/MyDrive/data/fr_1000.txt")
        self.fr_3000_list = self.__load("/content/drive/MyDrive/data/fr_3000.txt")
        self.fr_5000_list = self.__load("/content/drive/MyDrive/data/fr_5000.txt")
        self.fr_10000_list = self.__load("/content/drive/MyDrive/data/fr_10000.txt")
        self.fr_more_than_5list = self.__load("/content/drive/MyDrive/data/fr_more_than_5ipm.txt")
        self.fr_spoken_list = self.__load("/content/drive/MyDrive/data/fr_spoken.txt")

        # списки слов
        self.simple_russian_850_list = self.__load("/content/drive/MyDrive/data/SimpleRussian850.txt")
        self.simple_russian_1000_list = self.__load("/content/drive/MyDrive/data/simple_russian.txt")
        self.simple_russian_2000_list = self.__load("/content/drive/MyDrive/data/SimpleRussian2000.txt")
        self.brown_russian_10000_list = self.__load("/content/drive/MyDrive/data/Brown10000.txt")
        self.dale_russian_3000_list = self.__load("/content/drive/MyDrive/data/DaleRussian3000.txt")
        self.stop_list = self.__load("/content/drive/MyDrive/data/stop_list.txt")

        # семантические списки
        self.lex_abstract_list = self.__load("/content/drive/MyDrive/data/lex_abstract.txt")

        # списки слов для родного
        self.laposhina_list = self.__load("/content/drive/MyDrive/data/laposhina_list_from_formula.txt")
        self.detcorpus_list = self.__load("/content/drive/MyDrive/data/detcorpus_5000.txt")
        self.rki_children_1000 = self.__load("/content/drive/MyDrive/data/children_list_1000.txt")
        self.rki_children_2000 = self.__load("/content/drive/MyDrive/data/children_list_2000.txt")
        self.rki_children_5000 = self.__load("/content/drive/MyDrive/data/children_list_5000.txt")

        # считали датафрейм
        self.corpus_rnc = pd.read_csv("/content/drive/MyDrive/data/freq_rnc.csv", quotechar="`")
        self.lemmas_list_rnc = list(self.corpus_rnc["lemma"])

        # создаем словарь и будем в него все складывать
        self.data_about_text = {}
        self.whole_lemmas_list = []
        self.noun_list = []
        self.bastard_list = []
        self.obsc_list = []
        self.razgovor_list = []
        self.names_list = []
        self.geo_name_list = []
        self.conj_adversative_list = []  # противительные союзы
        self.modal_words_list = []
        self.words_length_list = []
        self.number_of_syllables_list = []
        self.long_words_list = []  # слова более чем из 4 слогов
        self.long_words_len_list = []
        self.count_kotoryi = []
        self.count_content_pos = []
        self.count_passive = []

        # Обучаем модель
        # x_train, y_train = self.features[ ], self.features[ ]
        # self.ridge.fit(x_train, y_train)
        # json.dump(self.ridge, "data/model.joblib")

        # Загружаем уже обученную модель
        # self.ridge = load(" ")

    def __load(self, file_name):
        f = open(file_name, "r", encoding="utf_8")
        lines = f.readlines()
        f.close()
        return [line.replace("\n", "") for line in lines]

    def __clean_text(self, input_text):
        new_text = re.sub(r"[-()\"#/@;:<>{}=~|]•", "", input_text)
        new_text = new_text.replace("­\n", "")
        new_text = new_text.replace("\n", " ")
        new_text = new_text.replace("•", "")
        new_text = new_text.replace("…", ".")
        new_text = new_text.replace("...", ".")
        new_text = new_text.replace("..", ".")
        new_text = new_text.replace("?.", "?")
        new_text = new_text.replace("!.", "!")
        new_text = new_text.replace("_", "")
        new_text = new_text.replace("!—", "! —")
        new_text = new_text.replace("?—", "? —")
        new_text = new_text.replace("\xad", "")
        return new_text    
       
    # Подсчитываем слоги и буквы
    SYLLABLES = ["а", "е", "ё", "и", "о", "у", "ы", "ю", "я", "э"]    
    TWO_VOWELS = ["ау", "ая", "аэ", "аю", "еа", "ее", "ею", "ея", "ие", "ии", "ио", "иу", "ию", "ия", "ое", "ою", "ую", "юю", "эт", "ые", "яя", "ье", "ья"]
    
    def __count_syllables(self, element):
        i_text = element.get("text")
        i_text_syl_counter = 0
        for ii in i_text:
            if ii in Analyzer.SYLLABLES:
                i_text_syl_counter += 1
        for i in Analyzer.TWO_VOWELS:
            if i_text.find(i) != -1:
                i_text_syl_counter -= 1
        if i_text == "его":
            i_text_syl_counter = 1
        if i_text_syl_counter == 0:
            i_text_syl_counter = 1
        self.number_of_syllables_list.append(i_text_syl_counter)
        self.words_length_list.append(len(i_text))
        if i_text_syl_counter >= 4:
            self.long_words_len_list.append(i_text_syl_counter)
            self.long_words_list.append(i_text)
        return True

    # Убираем имена, геообъекты и бастарды
    MODAL_WORDS = [ "хочется", "нужно", "надо", "кажется", "казаться", "пожалуй", "хотеть", "должный", "хотется"]

    def __clean_from_name_geo_bastard(self, element):
        self.whole_lemmas_list.append(element.get("analysis")[0]["lex"])
        gr_info = element.get("analysis")[0]["gr"]
        if "qual" in element.get("analysis")[0]:
            if element.get("analysis")[0]["qual"] == "bastard":
                self.bastard_list.append(element.get("text"))
        if gr_info.find("гео") > 0:
            self.geo_name_list.append(element.get("analysis")[0]["lex"])
        if gr_info.find("обсц") > 0:
            self.obsc_list.append(element.get("analysis")[0]["lex"])
        if gr_info.find("разг") > 0:
            self.razgovor_list.append(element.get("analysis")[0]["lex"])
        if (
            gr_info.find("имя") > 0
            or gr_info.find("фам") > 0
            or gr_info.find("отч") > 0
        ):
            self.names_list.append(element.get("analysis")[0]["lex"])
        if (
            element.get("analysis")[0]["lex"] == "но"
            or element.get("analysis")[0]["lex"] == "а"
            or element.get("analysis")[0]["lex"] == "однако"
            or element.get("analysis")[0]["lex"] == "зато"
        ):
            self.conj_adversative_list.append(element.get("analysis")[0]["lex"])
        if element.get("analysis")[0]["lex"] in Analyzer.MODAL_WORDS:
            self.modal_words_list.append(element.get("analysis")[0]["lex"])
        if element.get("analysis")[0]["lex"] == "который":
            self.count_kotoryi.append(element.get("analysis")[0]["lex"])
        return True

    # подсчет грам. информации
    def __count_gram(self, element):
        gr_info = element.get("analysis")[0]["gr"]
        gr_info = gr_info.replace(",", "<b>")
        gr_info = gr_info.replace("=", "<b>")
        gr_info = gr_info.split("<b>")
        for i in Analyzer.GRAM_FEATURES:
            if i in gr_info:
                self.dict_of_features[i] += 1
        if "S" in gr_info:
            self.noun_list.append(element.get("analysis")[0]["lex"])
        if "S" in gr_info or "V" in gr_info or "A" in gr_info or "ADV" in gr_info:
            self.count_content_pos.append(element.get("analysis")[0]["lex"])
        return True

    def __count_passive_form(self, element):
        if element[0].get("analysis") and element[1].get("analysis"):
            element0_gr = self.__get_gr_info(element[0])
            element1_gr = self.__get_gr_info(element[1])
            if element[0].get("analysis")[0]["lex"] == "быть" and "прош" in element0_gr:
                if "прич" in element1_gr:
                    self.count_passive.append(element[1].get("text"))
        return True

    def __get_gr_info(self, element):
        gr_info = element.get("analysis")[0]["gr"]
        gr_info = gr_info.replace(",", "<b>")
        gr_info = gr_info.replace("=", "<b>")
        gr_info = gr_info.split("<b>")
        return gr_info

    # Общий цикл просмотра анализа слов
    def __gram_analyze(self, element):
        for i in element:
            self.__count_syllables(i)
            if len(i.get("analysis")) > 0:
                self.__clean_from_name_geo_bastard(i)
                self.__count_gram(i)
        return True

    # Вычисляем процент слов из разных словников и частотных списков

    def __percent_of_known_words_100(self, element, list_of_words):
        if len(list_of_words) == 0 or len(element) == 0:
            return 0
        else:
            known_words = [w for w in element if w in list_of_words]
            percent = round((len(known_words) / len(element)) * 100)
            return percent

    def __get_frequency_bag(self, element):
        frequency_bag = dict()
        unique_lemmas = sorted(list(set(element)))
        for i in unique_lemmas:
            frequency_bag[i] = element.count(i)
        sorted_fr_bag = sorted(frequency_bag.items(), key=lambda x: x[1], reverse=True)
        return sorted_fr_bag

    # делим текст на предложения
    def __sent_tokenize_plus(self, this_text):
        new_text = this_text.replace("(с.", "(стр ")
        new_text = new_text.replace("на с.", "на стр")
        new_text = new_text.replace(".—", ". —")
        new_text = re.sub(r"([a-zа-я1-9])\.([A-ZА-Я])", "\\1. \\2", new_text)
        new_text = re.sub(r"([a-zа-я1-9])\!([A-ZА-Я])", "\\1! \\2", new_text)
        new_text = re.sub(r"([a-zа-я1-9])\?([A-ZА-Я])", "\\1? \\2", new_text)
        new_text = re.sub(r"(рис. )([1-9])", "рис \\2", new_text)
        # В г. Смоленске
        new_text = re.sub(r"( г. )([A-ZА-Я]{1})", " г<dot> \\2", new_text)
        # С.В. Морозов
        new_text = re.sub(
            r"([А-Я]{1}\.[А-Я]{1})\. ([A-ZА-Я]{1}[a-zа-я]+)", "\\1<dot> \\2", new_text
        )
        # достигает 55 см. в длину
        new_text = re.sub(r"( см. )([a-zа-я1-9])", " см<dot> \\2", new_text)

        sentences = sent_tokenize(new_text)

        new_sentences = []
        for i in sentences:
            # У лисички длина 3 м. А у котика - 2.
            if re.findall(r"([a-zа-я1-9])\. ([А-Я]{1})", i):
                i_new = re.sub(r"([a-zа-я1-9])\. ([А-Я]{1})", "\\1.<stop>\\2", i)
                i_split = i_new.split("<stop>")
                for ii in i_split:
                    new_sentences.append(ii)
                continue
            # И вы вырастили мух?- Нет пока.
            if re.findall(r"([a-zа-я1-9]\.|\?|\!)(- [А-Я]{1})", i):
                i_new = re.sub(r"([a-zа-я1-9]\.|\?|\!)(- [А-Я]{1})", "\\1<stop>\\2", i)
                i_split = i_new.split("<stop>")
                for ii in i_split:
                    new_sentences.append(ii)
                continue
            if re.findall(r"[а-яА-ЯёЁ]+", i):
                new_sentences.append(i)

        for i in new_sentences:
            i = i.replace("<dot>", ".")
        return new_sentences

    def __clear_fields(self):

        self.data_about_text = {}
        self.whole_lemmas_list = []
        self.noun_list = []
        self.bastard_list = []
        self.obsc_list = []
        self.names_list = []
        self.geo_name_list = []
        self.conj_adversative_list = []  # противительные союзы
        self.modal_words_list = []
        self.words_length_list = []
        self.number_of_syllables_list = []
        self.long_words_list = []  # слова более чем из 4 слогов
        self.long_words_len_list = []
        self.count_kotoryi = []
        self.count_content_pos = []
        self.count_passive = []
        return True

    def __first_check_len_text(self, element):

        # первая проверка текста - не слишком маленький
        if len(element) < 10:
            self.data_about_text["text_ok"] = False
            self.data_about_text[
                "text_error_message"
            ] = "Введите текст на русском языке не менее 5 слов."
            return self.data_about_text

        # Вторая проверка текста - не слишком большой
        if len(element) > 100000:
            self.data_about_text["text_ok"] = False
            self.data_about_text[
                "text_error_message"
            ] = "Введите текст не более 100 000 знаков."
            return self.data_about_text

        self.data_about_text["text_ok"] = True
        self.data_about_text["text_error_message"] = ""

        return self.data_about_text

    def __second_check_len_text(self, element):
        if len(element) < 5:
            self.data_about_text["text_ok"] = False
            self.data_about_text[
                "text_error_message"
            ] = "Введите текст на русском языке не менее 5 слов."
            return self.data_about_text

        self.data_about_text["text_ok"] = True
        self.data_about_text["text_error_message"] = ""
        return self.data_about_text

    # Корректный вывод !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    def __output_ball(self, ball):
        ball_str = ""
        if ball < 10:
            last_number = ball
            if last_number == 1:
                ball_str = "балл"
            if 2 <= last_number <= 4:
                ball_str = "балла"
            if 5 <= last_number <= 9:
                ball_str = "баллов"
        else:
            last_number = ball % 10
            if last_number == 1:
                ball_str = "балл"
            if 2 <= last_number <= 4:
                ball_str = "балла"
            if 5 <= last_number <= 9 or last_number == 0:
                ball_str = "баллов"
        return ball_str

    def start_native(self, raw_text):
        self.__clear_fields()

        self.raw_text = raw_text

        text = self.__clean_text(raw_text)

        # создаем словарь и будем в него все складывать
        self.dict_of_features = {}

        # чистый финальный списочек параметров
        self.data_about_text = {}

        # первая проверка текста на длину
        self.data_about_text = self.__first_check_len_text(text)

        if self.data_about_text["text_ok"] is False:
            return self.data_about_text

        self.sentences = self.__sent_tokenize_plus(text)
        self.whole_analyzed_text = self.mystem.analyze(text)  # весь текст одним списком
        analyzed_bigrams = list(nltk.bigrams(self.whole_analyzed_text))  # биграммочки

        for i in Analyzer.GRAM_FEATURES:
            self.dict_of_features[i] = 0

        # запускаем функцию со всеми грам. анализами
        self.__gram_analyze(self.whole_analyzed_text)

        for i in analyzed_bigrams:
            self.__count_passive_form(i)

        # вторая проверка, не менее 5 слов
        self.data_about_text = self.__second_check_len_text(self.whole_lemmas_list)
        if self.data_about_text["text_ok"] is False:
            return self.data_about_text

        self.clean_lemmas_list = [
            f
            for f in self.whole_lemmas_list
            if f not in self.geo_name_list
            and f not in self.names_list
            and f not in self.bastard_list
            and f not in self.stop_list
        ]

        all_words = len(self.whole_analyzed_text)
        all_sentences = len(self.sentences)
        all_syllables = sum(self.number_of_syllables_list)

        all_len_words = [len(f) for f in self.whole_lemmas_list]
        all_len_sentences = [len(f.split(" ")) for f in self.sentences]

        long_words = len(self.long_words_list)
        whole_lemmas_minus_stop = [
            f for f in self.whole_lemmas_list if f not in self.stop_list
        ]
        self.unique_lemmas_list = list(set(whole_lemmas_minus_stop))

        # Частотник по тексту
        self.dict_of_features["frequency_bag"] = self.__get_frequency_bag(self.whole_lemmas_list)

        # Цифры про текст:
        # всего слов в тексте
        self.dict_of_features["words"] = len(self.whole_analyzed_text)
        self.dict_of_features["characters"] = len(self.raw_text)
        self.dict_of_features["syllables"] = all_syllables
        self.dict_of_features["unique_words"] = len(self.unique_lemmas_list)
        # всего предложений в тексте
        self.dict_of_features["sentences"] = all_sentences
        # средняя длина слова в тексте
        self.dict_of_features["mean_len_word"] = round(
            ((sum(self.words_length_list)) / all_words), 1
        )
        self.dict_of_features["median_len_word"] = statistics.median(all_len_words)
        self.dict_of_features["median_len_sentence"] = statistics.median(
            all_len_sentences
        )

        # средняя длина предложения в тексте
        self.dict_of_features["mean_len_sentence"] = round(
            (all_words / all_sentences), 1
        )
        self.dict_of_features["mean_len_word_in_syllables"] = all_syllables / all_words
        self.dict_of_features["percent_of_long_words"] = long_words / all_words

        # lexical density - лексическая плотность, соотношение смысловых и служебных
        # частей речи: чем она выше, тем считается что текст сложнее
        self.dict_of_features[
            "lex_density"
        ] = f"{round((len(self.count_content_pos) / len(self.whole_lemmas_list)) * 10)} из 10"

        # type-token ratio (lexical diversity) - number of types/the number of tokens:
        # чем выше, тем лексика в тексте "однотипнее"
        # потом попробовать sttr: то же самое на отрезках в 1000 слов.
        # standardised type/token ratio
        self.dict_of_features["tt_ratio"] = round(
            ((len(self.unique_lemmas_list) / len(self.whole_lemmas_list))), 2
        )

        self.dict_of_features["passive"] = len(self.count_passive)

        # формулы читабельности (адаптированные, из диссера Оборневой)##
        formula_f_oborneva_genuine = round(206.835 - (60.1 * (all_syllables / all_words)) - (1.3 * (all_words / all_sentences)))

        formula_f_oborneva = formula_f_oborneva_genuine

        if formula_f_oborneva > 100:
            formula_f_oborneva = 100
        if formula_f_oborneva < 0:
            formula_f_oborneva = 0

        self.dict_of_features["formula_flesh_oborneva"] = f"{formula_f_oborneva} из 100 (чем больше - тем текст легче)"

        formula_f_k_oborneva = round(0.5 * (all_words / all_sentences) + 8.4 * (all_syllables / all_words) - 15.59)

        if formula_f_k_oborneva < 0:
            formula_f_k_oborneva = 0

        self.dict_of_features["formula_flesh_kinc_oborneva"] = (f"{formula_f_k_oborneva} (примерно должна ""соответствовать школьному классу)")

        in_laposhina_list = self.__percent_of_known_words_100(self.whole_lemmas_list, self.laposhina_list)

        self.dict_of_features["laposhina_list"] = f"{in_laposhina_list} %"

        in_detcorpus_5000 = self.__percent_of_known_words_100(
            self.whole_lemmas_list, self.detcorpus_list
        )

        self.dict_of_features["detcorpus_5000"] = f"{in_detcorpus_5000} %"

        self.dict_of_features["rare_words"] = list(
            set(
                [
                    f
                    for f in self.clean_lemmas_list
                    if f not in self.detcorpus_list and f not in self.fr_10000_list
                ]
            )
        )

        structure_complex_genuine = round((100 - formula_f_oborneva_genuine + self.dict_of_features["прич"] + self.dict_of_features["страд"] + self.dict_of_features["passive"]) / 10)

        structure_complex = structure_complex_genuine

        if structure_complex < 0:
            structure_complex = 0
        if structure_complex > 10:
            structure_complex = 10

        self.dict_of_features["structure_complex"] = f"{structure_complex} из 10"

        lexical_complex_genuine = round(10 - (((in_laposhina_list - 50) * 2) / 10))

        lexical_complex = lexical_complex_genuine

        if lexical_complex > 10:
            lexical_complex = 10
        if lexical_complex < 0:
            lexical_complex = 0

        self.dict_of_features["lexical_complex"] = f"{lexical_complex} из 10"

        lexical_complex_rki = round(10 - ((((in_detcorpus_5000 - 60) * 2)) / 10))

        if lexical_complex_rki > 10:
            lexical_complex_rki = 10
        if lexical_complex_rki < 0:
            lexical_complex_rki = 0

        self.dict_of_features["lexical_complex_rki"] = f"{lexical_complex_rki} из 10"

        #Повествовательность
        narrativity = round(10 - 2 * (self.dict_of_features["S"] / (self.dict_of_features["V"] + 1)))

        if narrativity < 0:
            narrativity = 0

        self.dict_of_features["narrativity"] = f"{narrativity} из 10"
        

        # Описательность
        description = round(3 * (self.dict_of_features["A"] / all_sentences))

        if description > 10:
            description = 10

        self.dict_of_features["description"] = f"{description} из 10"

        
        # Вычисляем и выводим формулу Пушкина
        formula_pushkin_100 = round((((structure_complex_genuine + lexical_complex_genuine) * 5) - narrativity), 1,)

        if formula_pushkin_100 > 100:
            formula_pushkin_100 = 99
        if formula_pushkin_100 < 1:
            formula_pushkin_100 = 1

        self.dict_of_features["formula_pushkin_100"] = round(formula_pushkin_100)

        self.dict_of_features["formula_pushkin"] = round((formula_pushkin_100 / 10), 1)

        if self.dict_of_features["formula_pushkin"] < 1:
            self.dict_of_features["formula_pushkin"] = 1

        ball = self.__output_ball(formula_pushkin_100)

        for i in Analyzer.INTERPRETER:
            if i[1] < formula_pushkin_100 <= i[2]:
                self.dict_of_features["level_comment"] = f"{round(formula_pushkin_100)} {ball} из 100. {i[0]}"

        for i in self.dict_of_features:
            if i in Analyzer.COLUMNS_NEEDED_NATIVE:
                self.data_about_text[i] = self.dict_of_features[i]

        return self.data_about_text

In [ ]:
# Оценка текста
analyzer = Analyzer(mystem)
analyzer.start_native(raw)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Ridge from version 0.23.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'text_ok': True,
 'text_error_message': '',
 'frequency_bag': [('по', 92),
  ('рейтинговый', 70),
  ('шкала', 70),
  ('для', 67),
  ('либо', 60),
  ('российский', 47),
  ('федерация', 46),
  ('национальный', 44),
  ('в', 33),
  ('пункт', 28),
  ('с', 28),
  ('международный', 26),
  ('положение', 26),
  ('ао', 23),
  ('выпуск', 22),
  ('соответствие', 20),
  ('ооо', 18),
  ('подпункт', 16),
  ('облигация', 15),
  ('акр', 13),
  ('эксперт', 13),
  ('абзац', 12),
  ('гарант', 11),
  ('поручитель', 11),
  ('актив', 10),
  ('кредитный', 9),
  ('нра', 9),
  ('рейтинг', 9),
  ('выплата', 8),
  ('от', 8),
  ('заем', 7),
  ('и', 7),
  ('который', 7),
  ('облигационный', 7),
  ('субординированный', 7),
  ('за', 6),
  ('зависеть', 6),
  ('изменение', 6),
  ('исключение', 6),
  ('стоимость', 6),
  ('указывать', 6),
  ('лицо', 5),
  ('отсутствие', 5),
  ('первый', 5),
  ('случай', 5),
  ('требование', 5),
  ('третий', 5),
  ('устанавливать', 5),
  ('вид', 4),
  ('или', 4),
  ('ипотечный', 4),
  ('

In [ ]:
# Текст
raw = 'Совет директоров Банка России 23 декабря 2022 года принял решение: 1. В целях применения Положения Банка России от 16.11.2021 № 781-П «О требованиях к финансовой устойчивости и платежеспособности страховщиков» (далее — Положение) установить следующие уровни кредитных рейтингов: 1.1. В отношении российских объектов рейтинга: в соответствии с абзацем седьмым пункта 2.6 и абзацем третьим подпункта 3.1.12.4 пункта 3.1 Положения для кредитной организации, в которой учитываются полученные от страховой организации денежные средства на специальном счете — «ruB+» по национальной рейтинговой шкале для Российской Федерации АО «Эксперт РА», либо «B+(RU)» по национальной рейтинговой шкале для Российской Федерации АКРА (АО), либо «B+.ru» по национальной рейтинговой шкале для Российской Федерации ООО «НКР», либо «B+|ru|» по национальной рейтинговой шкале для Российской Федерации ООО «НРА»;в соответствии с абзацем первым подпункта 3.1.8 пункта 3.1 Положения для выпусков облигаций (за исключением выпусков облигаций, выплаты (часть выплат) по которым установлены в виде формулы с переменными и (или) зависят от изменения стоимости активов (за исключением активов, указанных в пункте 3.7 Положения), включая выпуски облигаций с ипотечным покрытием, выпусков субординированных облигационных займов) — «ruB+» по национальной рейтинговой шкале для Российской Федерации АО «Эксперт РА», либо «ruВ+.sf» по национальной рейтинговой шкале для Российской Федерации, применяемой для присвоения кредитных рейтингов инструментам структурированного финансирования АО «Эксперт РА», либо «B+(RU)» по национальной рейтинговой шкале для Российской Федерации АКРА (АО), либо «В+(ru.sf)» по национальной рейтинговой шкале сектора структурированного финансирования для Российской Федерации АКРА (АО);в соответствии с абзацем первым подпункта 3.1.8 пункта 3.1 Положения для эмитентов либо поручителей (гарантов) по выпускам облигаций (за исключением выпусков облигаций, выплаты (часть выплат) по которым установлены в виде формулы с переменными и (или) зависят от изменения стоимости активов (за исключением активов, указанных в пункте 3.7 Положения), включая выпуски облигаций с ипотечным покрытием, выпуски субординированных облигационных займов), в случае отсутствия кредитного рейтинга выпуска облигаций — «ruB+» по национальной рейтинговой шкале для Российской Федерации АО «Эксперт РА», либо «B+(RU)» по национальной рейтинговой шкале для Российской Федерации АКРА (АО), либо «B+.ru» по национальной рейтинговой шкале для Российской Федерации ООО «НКР», либо «B+|ru|» по национальной рейтинговой шкале для Российской Федерации ООО «НРА»;в соответствии с абзацем первым подпункта 3.1.8 пункта 3.1 Положения для выпусков облигаций, выплаты (часть выплат) по которым установлены в виде формулы с переменными и (или) зависят от изменения стоимости активов (за исключением активов, указанных в пункте 3.7 Положения), в том числе выпусков облигаций с ипотечным покрытием — «ruAAA.sf» по национальной рейтинговой шкале для Российской Федерации, применяемой для присвоения кредитных рейтингов инструментам структурированного финансирования АО «Эксперт РА» либо «ААА(ru.sf)» по национальной рейтинговой шкале сектора структурированного финансирования для Российской Федерации АКРА (АО);в соответствии с абзацем первым подпункта 3.1.8 пункта 3.1 Положения для эмитентов либо поручителей (гарантов) по выпускам облигаций, выплаты (часть выплат) по которым установлены в виде формулы с переменными и (или) зависят от изменения стоимости активов (за исключением активов, указанных в пункте 3.7 Положения), в том числе по выпускам облигаций с ипотечным покрытием, в случае отсутствия кредитного рейтинга выпуска облигаций — «ruААА» по национальной рейтинговой шкале для Российской Федерации АО «Эксперт РА», либо «ААА(RU)» по национальной рейтинговой шкале для Российской Федерации АКРА (АО), либо «AAA.ru» по национальной рейтинговой шкале для Российской Федерации ООО «НКР», либо «ААА|ru|» по национальной рейтинговой шкале для Российской Федерации ООО «НРА»;в соответствии с подпунктом 3.1.9 пункта 3.1 Положения для выпусков субординированных облигационных займов — «ruB+» по национальной рейтинговой шкале для Российской Федерации АО «Эксперт РА» либо «B+(RU)» по национальной рейтинговой шкале для Российской Федерации АКРА (АО);в соответствии с подпунктом 3.1.9 пункта 3.1 Положения для эмитентов либо поручителей (гарантов) по выпускам субординированных облигационных займов, в случае отсутствия рейтинга по выпускам субординированных облигационных займов — «ruA-» по национальной рейтинговой шкале для Российской Федерации АО «Эксперт РА», либо «A-(RU)» по национальной рейтинговой шкале для Российской Федерации АКРА (АО), либо «A-.ru» по национальной рейтинговой шкале для Российской Федерации ООО «НКР», либо «А-|ru|» по национальной рейтинговой шкале для Российской Федерации ООО «НРА»;в соответствии с подпунктом 3.1.12.1 пункта 3.1 Положения для должника либо поручителя (гаранта) по правам требования по денежному обязательству — «ruB+» по национальной рейтинговой шкале для Российской Федерации АО «Эксперт РА», либо «B+(RU)» по национальной рейтинговой шкале для Российской Федерации АКРА (АО), либо «B+.ru» по национальной рейтинговой шкале для Российской Федерации ООО «НКР», либо «B+|ru|» по национальной рейтинговой шкале для Российской Федерации ООО «НРА»;в соответствии с подпунктом 3.1.12.6 пункта 3.1 Положения для контрагента либо поручителя (гаранта) по правам требования по договору репо — «ruB+» по национальной рейтинговой шкале для Российской Федерации АО «Эксперт РА», либо «B+(RU)» по национальной рейтинговой шкале для Российской Федерации АКРА (АО), либо «B+.ru» по национальной рейтинговой шкале для Российской Федерации ООО «НКР», либо «B+|ru|» по национальной рейтинговой шкале для Российской Федерации ООО «НРА»;в соответствии с абзацем пятым пункта 3.3 Положения для контрагента либо поручителя (гаранта) по срочной сделке — «ruB+» по национальной рейтинговой шкале для Российской Федерации АО «Эксперт РА», либо «B+(RU)» по национальной рейтинговой шкале для Российской Федерации АКРА (АО), либо «B+.ru» по национальной рейтинговой шкале для Российской Федерации ООО «НКР», либо «B+|ru|» по национальной рейтинговой шкале для Российской Федерации ООО «НРА»;в соответствии с абзацем тринадцатым пункта 3.3 Положения для третьих лиц — «ruB+» по национальной рейтинговой шкале для Российской Федерации АО «Эксперт РА», либо «B+(RU)» по национальной рейтинговой шкале для Российской Федерации АКРА (АО), либо «B+.ru» по национальной рейтинговой шкале для Российской Федерации ООО «НКР», либо «B+|ru|» по национальной рейтинговой шкале для Российской Федерации ООО «НРА»;в соответствии с пунктом 3.7 Положения для третьих лиц, от исполнения обязательств которыми зависит изменение стоимости активов, указанных в данном пункте — «ruB+» по национальной рейтинговой шкале для Российской Федерации АО «Эксперт РА», либо «B+(RU)» по национальной рейтинговой шкале для Российской Федерации АКРА (АО), либо «B+.ru» по национальной рейтинговой шкале для Российской Федерации ООО «НКР», либо «B+|ru|» по национальной рейтинговой шкале для Российской Федерации ООО «НРА».1.2. В отношении иностранных объектов рейтинга:в соответствии с абзацем первым подпункта 3.1.8 пункта 3.1 Положения для выпусков облигаций, для эмитентов либо поручителей (гарантов) по выпускам облигаций, в случае отсутствия кредитного рейтинга выпусков облигаций — «B-» по международной рейтинговой шкале S&P Global Ratings, либо «B3» по международной рейтинговой шкале Moody’s Investors Service, либо «B-» по международной рейтинговой шкале Fitch Ratings; соответствии с подпунктом 3.1.9 пункта 3.1 Положения для поручителей (гарантов) по выпускам субординированных облигационных займов, в случае отсутствия кредитного рейтинга по выпускам субординированных облигационных займов — «BB-» по международной рейтинговой шкале S&P Global Ratings, либо «Ba3» по международной рейтинговой шкале Moody’s Investors Service, либо «BB-» по международной рейтинговой шкале Fitch Ratings;в соответствии с подпунктом 3.1.12.1 пункта 3.1 Положения для должника либо поручителя (гаранта) по правам требования по денежному обязательству — «B-» по международной рейтинговой шкале S&P Global Ratings, либо «B3» по международной рейтинговой шкале Moody’s Investors Service, либо «B-» по международной рейтинговой шкале Fitch Ratings, либо «C++» по международной рейтинговой шкале A.M. Best Co;в соответствии с подпунктом 3.1.12.6 пункта 3.1 Положения для контрагента либо поручителя (гаранта) по правам требования по договору репо — «B-» по международной рейтинговой шкале S&P Global Ratings, либо «B3» по международной рейтинговой шкале Moody’s Investors Service, либо «B-» по международной рейтинговой шкале Fitch Ratings;в соответствии с подпунктом 3.1.12.9 и абзацем четвертым подпункта 3.1.12.13 пункта 3.1 Положения для перестраховщиков, не являющихся резидентами Российской Федерации, — «B» по международной рейтинговой шкале S&P Global Ratings, либо «B2» по международной рейтинговой шкале Moody’s Investors Service, либо «B» по международной рейтинговой шкале Fitch Ratings, либо «B-» по международной рейтинговой шкале A.M. Best Co;в соответствии с абзацем пятым пункта 3.3 Положения для контрагента либо поручителя (гаранта) по срочной сделке — «B-» по международной рейтинговой шкале S&P Global Ratings, либо «B3» по международной рейтинговой шкале Moody’s Investors Service, либо «B-» по международной рейтинговой шкале Fitch Ratings;в соответствии с абзацем тринадцатым пункта 3.3 Положения для третьих лиц — «B-» по международной рейтинговой шкале S&P Global Ratings, либо «B3» по международной рейтинговой шкале Moody’s Investors Service, либо «B-» по международной рейтинговой шкале Fitch Ratings;в соответствии с пунктом 3.7 Положения для третьих лиц, от исполнения обязательств которыми зависит изменение стоимости активов, указанных в данном пункте — «B-» по международной рейтинговой шкале S&P Global Ratings, либо «B3» по международной рейтинговой шкале Moody’s Investors Service, либо «B-» по международной рейтинговой шкале Fitch Ratings.3. В отношении определения групп кредитного качества, указанных в подпункте 6.5.2.2 пункта 6.5 Положения:Для лиц, являющихся резидентами Российской Федерации:'